# Ensemble Learning through Batch Boosting


In [87]:
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

# add project root folder to path to allow import local modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import local modules
from stochastic_models import *
from visualisations import *

## Parameter Configuration

In [88]:
np.random.seed(2020)

In [89]:
N_train = 5000
N_test = 50000
Epsilon  = 1e-6
Max_Iter = 50
Learning_Rate = 0.366
Ratio = 0.4
ratios = [0.15, 0.2, 0.25, 0.35, 0.4, 0.5]
d = 1
T = 2

In [90]:
lambda_range = (N_train*1e-9 , N_train*1e-3)
alpha_range = (8.3*1e-5, 0.83)
length_scale = np.sort(1/np.sqrt((2*alpha_range[0], 2*alpha_range[1])))

## Create Kernel & prepare datasets

In [91]:
#kernel
kernel = RBF(length_scale= (length_scale[0] + length_scale[1])/2, length_scale_bounds=length_scale) \
        + WhiteKernel(noise_level= (lambda_range[0] + lambda_range[1])/2 , noise_level_bounds=lambda_range)


In [92]:
#generate drivers
s_train = MaxCallStochasticModel(N_train,d,[1/12,11/12])
s_train.generate_samples()
s_test = MaxCallStochasticModel(N_test, d, [1/12,11/12])
s_test.generate_samples()

In [93]:
y_train = s_train.y
X_train = s_train.X
S_train = s_train.S

y_test = s_test.y
X_test = s_test.X
S_test = s_test.S

V_T = s_test.generate_true_V(T)
V_0 = s_test.generate_true_V(0)

V_T_train = s_train.generate_true_V(T)
V_0_train = s_train.generate_true_V(0)

V_0_train = V_0_train.mean()
V_0= V_0.mean()
V_0 # Average expected PROFIT!

0.07935344411249907

In [94]:
from aggregating.utils import flatten_X, normalized_error_VT
#dim(N_test,d*T)
Flattened_X_test = flatten_X(X_test)
# dim(N_train, d*T)
Flattened_X_train = flatten_X(X_train)

In [95]:
#Model with Flatten_X_1
m_1 = GaussianProcessRegressor(kernel)
m_1.fit(Flattened_X_train,y_train)

GaussianProcessRegressor(kernel=RBF(length_scale=39.2) + WhiteKernel(noise_level=2.5))

In [96]:
fX_1 = m_1.predict(Flattened_X_test)

In [97]:
fX_2 = m_1.predict(Flattened_X_train)

In [98]:
test_reference_error = normalized_error_VT(fX_1 ,V_T, V_0)

In [99]:
train_reference_error = normalized_error_VT(fX_2 ,V_T_train, V_0_train)

In [100]:
train_reference_error

0.020513219063138088

In [101]:
test_reference_error 

0.13165511169540253

## Stochastic Gradient Boosting

In [122]:
def Stochastic_Gradient_Boosting(X_train, y_train, X_test, y_test, Max_Iter, min_error, early_stop, kernel, learning_rate, epsilon, sample_size, init):
    np.random.seed(2020)
    error_going_up = 0
    train_prediction = np.zeros(N_train)
    test_prediction = np.zeros(N_test)
    previous_y_hat = np.zeros(N_train)
    previous_y_test_hat = np.zeros(N_test)
    models = []
    train_errors = []
    test_errors = []
    
    current_residual = np.full(len(y_train),init)
    y_hat_test = np.full(len(y_test),init)

   
    
    for i in range(Max_Iter):
        model = GaussianProcessRegressor(kernel)
    
        indices = np.random.choice(X_train.shape[0],size=sample_size,replace=False)
       
        model.fit(X_train[indices], y_train[indices] - current_residual[indices])
        models.append(model)
        
        new_train_predictor = model.predict(X_train)
        new_test_predictor = model.predict(X_test)
        
        current_residual += learning_rate * new_train_predictor
        
        y_hat_train = current_residual 
        
        y_hat_test += learning_rate * new_test_predictor
        
        #Evaluating TrainSet error
        
        train_errors.append(normalized_error_VT(y_hat_train,y_train,V_0_train).item())
        
        ##Evaluating TestSet error
        
        test_errors.append(normalized_error_VT(y_hat_test,y_test,V_0).item())
    
        if (( np.abs(min_error[0] - test_errors[-1]) > epsilon) & (test_errors[-1] < min_error[0])):
            min_error = (test_errors[-1], i)
            error_going_up = 0
            print('For iteration number {}, the test error decreased , test error : {} '.format(i, min_error[0]))
        else:
            error_going_up += 1
            print('For iteration number {}, the test error increased , test error : {} '.format(i, min_error[0]))
        
            if  (i==(Max_Iter-1)):
                models = models[:min_error[1]+1]
                
                return models, train_errors, test_errors, min_error
        
            elif (error_going_up == early_stop ):
                models = models[: -(early_stop)]
                
                return models, train_errors, test_errors, min_error
                break #early stopping
            
    return models, train_errors, test_errors, min_error

In [123]:
def Stoch_Grad_Boosting_per_ratio(ratios, model, X_train, y_train, X_test, y_test, Epsilon, kernel, Max_Iter, init):
    models = dict((val, []) for val in range(len(ratios))) 
    train_errors = dict((val, []) for val in range(len(ratios))) 
    test_errors = dict((val, []) for val in range(len(ratios))) 
    min_errors = dict((val, []) for val in range(len(ratios))) 

    for i in range(len(ratios)) :
        print(f"Starting evaluation for ratio {ratios[i]}:")
        models[i], train_errors[i], test_errors[i], min_errors[i] = model(X_train, y_train, X_test, y_test, Max_Iter, (float("inf"), float("inf")), 5, kernel, 0.366, Epsilon, round(X_train.shape[0]*ratios[i]), init)
    
    return test_errors


In [124]:
test_reference_error = 0.04973099137089055

In [ ]:
test_errors = Stoch_Grad_Boosting_per_ratio(ratios, Stochastic_Gradient_Boosting, np.copy(Flattened_X_train), np.copy(y_train), np.copy(Flattened_X_test), np.copy(y_test), Epsilon, kernel, Max_Iter, np.mean(y_train, axis=0))
test_errors_0 = Stoch_Grad_Boosting_per_ratio(ratios, Stochastic_Gradient_Boosting, np.copy(Flattened_X_train), np.copy(y_train), np.copy(Flattened_X_test), np.copy(y_test), Epsilon, kernel, Max_Iter, 0.0)
test_errors_1 = Stoch_Grad_Boosting_per_ratio(ratios, Stochastic_Gradient_Boosting, np.copy(Flattened_X_train), np.copy(y_train), np.copy(Flattened_X_test), np.copy(y_test), Epsilon, kernel, Max_Iter, np.mean(y_train, axis=0)/2)


Starting evaluation for ratio 0.15:
For iteration number 0, the test error decreased , test error : 1.051834041421147 
For iteration number 1, the test error decreased , test error : 0.6766337512718569 
For iteration number 2, the test error decreased , test error : 0.4410143410379095 
For iteration number 3, the test error decreased , test error : 0.29210934915803577 
For iteration number 4, the test error decreased , test error : 0.1982839332202317 
For iteration number 5, the test error decreased , test error : 0.14370820604078463 
For iteration number 6, the test error decreased , test error : 0.10277889444997268 
For iteration number 7, the test error decreased , test error : 0.0869429738408516 
For iteration number 8, the test error decreased , test error : 0.06754337293664624 
For iteration number 9, the test error decreased , test error : 0.06406612755801742 
For iteration number 10, the test error decreased , test error : 0.062298938945518585 
For iteration number 11, the test

/Users/Clement/anaconda3/lib/python3.7/site-packages/sklearn/gaussian_process/_gpr.py:504: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


For iteration number 8, the test error decreased , test error : 0.07116879649257232 
For iteration number 9, the test error decreased , test error : 0.06772443029836256 
For iteration number 10, the test error decreased , test error : 0.0663907361401279 
For iteration number 11, the test error decreased , test error : 0.06570823600496442 
For iteration number 12, the test error decreased , test error : 0.06530632594664988 
For iteration number 13, the test error decreased , test error : 0.06522307899337136 
For iteration number 14, the test error decreased , test error : 0.0650646572479231 
For iteration number 15, the test error decreased , test error : 0.06492816335186208 
For iteration number 16, the test error decreased , test error : 0.0648566589768507 
For iteration number 17, the test error increased , test error : 0.0648566589768507 
For iteration number 18, the test error decreased , test error : 0.06478654208514528 
For iteration number 19, the test error increased , test err

/Users/Clement/anaconda3/lib/python3.7/site-packages/sklearn/gaussian_process/_gpr.py:504: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


For iteration number 26, the test error increased , test error : 0.06473587986424427 
For iteration number 27, the test error increased , test error : 0.06473587986424427 
For iteration number 28, the test error increased , test error : 0.06473587986424427 
Starting evaluation for ratio 0.25:
For iteration number 0, the test error decreased , test error : 1.053667290628139 
For iteration number 1, the test error decreased , test error : 0.6771897283354777 
For iteration number 2, the test error decreased , test error : 0.4369716091641173 
For iteration number 3, the test error decreased , test error : 0.2851843836975467 
For iteration number 4, the test error decreased , test error : 0.19474671728254642 
For iteration number 5, the test error decreased , test error : 0.132173228245569 
For iteration number 6, the test error decreased , test error : 0.09890803876350772 
For iteration number 7, the test error decreased , test error : 0.08293316314080539 
For iteration number 8, the test 

...

/Users/Clement/anaconda3/lib/python3.7/site-packages/sklearn/gaussian_process/_gpr.py:504: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


For iteration number 5, the test error decreased , test error : 0.14818160732221144 
For iteration number 6, the test error decreased , test error : 0.10473134322448668 
For iteration number 7, the test error decreased , test error : 0.08784145903025749 
For iteration number 8, the test error decreased , test error : 0.0677648453105139 
For iteration number 9, the test error decreased , test error : 0.06400319893347996 
For iteration number 10, the test error decreased , test error : 0.06209061392742289 
For iteration number 11, the test error decreased , test error : 0.06129295730742922 
For iteration number 12, the test error decreased , test error : 0.06065587166215776 
For iteration number 13, the test error decreased , test error : 0.05412146226141979 
For iteration number 14, the test error decreased , test error : 0.052680920468395266 
For iteration number 15, the test error decreased , test error : 0.052610019788522026 


/Users/Clement/anaconda3/lib/python3.7/site-packages/sklearn/gaussian_process/_gpr.py:504: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


For iteration number 16, the test error increased , test error : 0.052610019788522026 
For iteration number 17, the test error increased , test error : 0.052610019788522026 
For iteration number 18, the test error increased , test error : 0.052610019788522026 
For iteration number 19, the test error decreased , test error : 0.05098682312004978 
For iteration number 20, the test error increased , test error : 0.05098682312004978 
For iteration number 21, the test error increased , test error : 0.05098682312004978 
For iteration number 22, the test error increased , test error : 0.05098682312004978 
For iteration number 23, the test error increased , test error : 0.05098682312004978 
For iteration number 24, the test error increased , test error : 0.05098682312004978 
Starting evaluation for ratio 0.2:
For iteration number 0, the test error decreased , test error : 1.0970716352632568 
For iteration number 1, the test error decreased , test error : 0.7041057614338119 
For iteration number

For iteration number 17, the test error decreased , test error : 0.10860607792330408 
For iteration number 18, the test error increased , test error : 0.10860607792330408 
For iteration number 19, the test error increased , test error : 0.10860607792330408 
For iteration number 20, the test error increased , test error : 0.10860607792330408 
For iteration number 21, the test error decreased , test error : 0.10851003896655724 
For iteration number 22, the test error increased , test error : 0.10851003896655724 
For iteration number 23, the test error decreased , test error : 0.10844722006705106 
For iteration number 24, the test error increased , test error : 0.10844722006705106 
For iteration number 25, the test error increased , test error : 0.10844722006705106 
For iteration number 26, the test error increased , test error : 0.10844722006705106 
For iteration number 27, the test error decreased , test error : 0.1084252152005595 
For iteration number 28, the test error decreased , tes

In [ ]:
plot_errors(test_errors,ratios, N_train, d, N_test, Learning_Rate, Max_Iter, 'mean') 
plot_errors(test_errors_0,ratios, N_train, d, N_test, Learning_Rate, Max_Iter, '0') 
plot_errors(test_errors_1,ratios, N_train, d, N_test, Learning_Rate, Max_Iter, 'mean/2') 

In [1]:
import pickle
PICKLE = False
if PICKLE:
    test_errors = pickle.load(open("mean_initalisation.p", "rb"))
    test_errors_0 = pickle.load(open("0_initalisation.p", "rb"))
    test_errors_1 = pickle.load(open("half_mean_initalisation.p", "rb"))

In [90]:
def log_plot_errors(test_errors, ratios, N_train, d, N_test, Learning_Rate, Max_Iter):
    for i in range(len(test_errors)):
        while len(test_errors[i]) != Max_Iter :
            test_errors[i].append(test_errors[i][-1])
    plt.figure(figsize=((18, 6)))
    for i in range(len(ratios)) :
        #plt.axvline(min_errors[i][1], ls='-', color = 'k', label = 'Test Minimal Error for rate : {rate}')
        plt.plot(np.arange(len(test_errors[i][4:61])) , np.log(test_errors[i][4:61]), label = f'learning rate :{ratios[i]}')
    plt.axhline(np.log(test_reference_error), ls=':', color = 'orange', label = 'Test Reference Error')
    plt.xlabel("Number of Iterations")
    plt.title(f"Stochastic Gradient Boosting: N_train = {N_train}, d= {d}, N_test= {N_test}, learning_rate : {Learning_Rate}, with different Learning Rates")
    plt.ylabel("Log Normalized Error")
    plt.legend(loc='upper right')
    plt.show()

In [6]:
def plot_errors(test_errors, ratios, N_train, d, N_test, Learning_Rate, Max_Iter, init):
    for i in range(len(test_errors)):
        while len(test_errors[i]) != Max_Iter :
            test_errors[i].append(test_errors[i][-1])
    plt.figure(figsize=((18, 6)))
    for i in range(len(ratios)) :
        #plt.axvline(min_errors[i][1], ls='-', color = 'k', label = 'Test Minimal Error for rate : {rate}')
        plt.plot(np.arange(7,26) , test_errors[i][7:26], label = f'alpha :{ratios[i]}')
    plt.axhline(test_reference_error, ls=':', color = 'orange', label = 'Test Reference Error')
    plt.xlabel("Number of Iterations")
    plt.title(f"Stochastic Gradient Boosting with {init} initialisation: N_train = {N_train}, d= {d}, N_test= {N_test}, learning_rate : {Learning_Rate}, with different Learning Rates")
    plt.ylabel("Normalized Error")
    plt.legend(loc='upper right')
    plt.show()